In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

In [2]:
# Read in CSV File
document_check = 'document_check_sample.csv'
facial_report = 'facial_report_sample.csv'

document_df = pd.read_csv(document_check)
facial_df = pd.read_csv(facial_report)


## Understand the attributes of data frames: 
- What are the attributes 
- What are their unique values 

In [3]:
# Understand the attributes
document_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11760 entries, 0 to 11759
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          5880 non-null   float64
 1   user_id                             5880 non-null   object 
 2   result                              5880 non-null   object 
 3   visual_authenticity_result          4991 non-null   object 
 4   image_integrity_result              5880 non-null   object 
 5   face_detection_result               4990 non-null   object 
 6   image_quality_result                5880 non-null   object 
 7   created_at                          5880 non-null   object 
 8   supported_document_result           5869 non-null   object 
 9   conclusive_document_quality_result  3140 non-null   object 
 10  colour_picture_result               3140 non-null   object 
 11  data_validation_result              4739 

In [4]:
# Understand the attributes
facial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11760 entries, 0 to 11759
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     5880 non-null   float64
 1   user_id                        5880 non-null   object 
 2   result                         5880 non-null   object 
 3   face_comparison_result         5535 non-null   object 
 4   created_at                     5880 non-null   object 
 5   facial_image_integrity_result  5861 non-null   object 
 6   visual_authenticity_result     4991 non-null   object 
 7   properties                     5880 non-null   object 
 8   attempt_id                     5880 non-null   object 
dtypes: float64(1), object(8)
memory usage: 827.0+ KB


## Dropping Nan vales 
- Based on the information above, there are 11760 entries, however there are only 5880 rows with result. Which means we canont make any assumption out of the rows with no result
- Hence, I will decide to drop all those rows

In [5]:
# Dropping the rows with Na values for result
document_df = document_df.dropna(subset=['result']).reset_index(drop=True)
facial_df = facial_df.dropna(subset=['result']).reset_index(drop=True)



## Handling document file

In [6]:
# Remmove unrelevant attributes

document_importantAttr = [
 'result', 
 'visual_authenticity_result',
 'image_integrity_result',
 'face_detection_result',
 'image_quality_result',
 'created_at',
 'supported_document_result',
 'conclusive_document_quality_result',
 'colour_picture_result',
 'data_validation_result',
 'data_consistency_result',
 'data_comparison_result',
 'police_record_result',
 'compromised_document_result',]

In [7]:
# Currently, the date time is object so it will not be sorted correctly. We will convert that to date-time format

document_df['created_at'] = pd.to_datetime(document_df['created_at'], format='%d/%m/%Y %H:%M')
document_df['created_at'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 5880 entries, 0 to 5879
Series name: created_at
Non-Null Count  Dtype         
--------------  -----         
5880 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 46.1 KB


In [8]:
# Since the result recently decreased, sort the dataset by date 'created_at' 

document_filtered = document_df[document_importantAttr]
document_filtered = document_filtered.sort_values(by='created_at').reset_index(drop=True)

In [9]:
# Check the unique values in all attributes 

for attribute in document_importantAttr:
    unique_values = document_filtered[attribute].unique()
    print(f"Unique values for {attribute}:\n{unique_values}\n")


Unique values for result:
['clear' 'consider']

Unique values for visual_authenticity_result:
['clear' 'consider' nan]

Unique values for image_integrity_result:
['clear' 'consider']

Unique values for face_detection_result:
['clear' nan 'consider']

Unique values for image_quality_result:
['clear' 'unidentified']

Unique values for created_at:
['2017-05-23T17:25:00.000000000' '2017-05-23T19:17:00.000000000'
 '2017-05-23T20:49:00.000000000' ... '2017-10-31T21:11:00.000000000'
 '2017-10-31T21:46:00.000000000' '2017-10-31T22:06:00.000000000']

Unique values for supported_document_result:
['clear' nan 'unidentified']

Unique values for conclusive_document_quality_result:
[nan 'clear' 'consider']

Unique values for colour_picture_result:
[nan 'clear' 'consider']

Unique values for data_validation_result:
['clear' nan 'consider']

Unique values for data_consistency_result:
[nan 'clear' 'consider']

Unique values for data_comparison_result:
['clear' 'consider' nan]

Unique values for police_

In [10]:
document_filtered

,result,visual_authenticity_result,image_integrity_result,face_detection_result,image_quality_result,created_at,supported_document_result,conclusive_document_quality_result,colour_picture_result,data_validation_result,data_consistency_result,data_comparison_result,police_record_result,compromised_document_result
0,clear,clear,clear,clear,clear,2017-05-23 17:25:00,clear,NaN,NaN,clear,NaN,clear,clear,NaN
1,clear,clear,clear,clear,clear,2017-05-23 19:17:00,clear,NaN,NaN,clear,clear,clear,clear,NaN
2,clear,clear,clear,clear,clear,2017-05-23 20:49:00,clear,NaN,NaN,clear,clear,clear,clear,NaN
3,consider,clear,clear,clear,clear,2017-05-24 07:47:00,clear,NaN,NaN,clear,clear,consider,clear,NaN
4,clear,clear,clear,clear,clear,2017-05-24 11:38:00,clear,NaN,NaN,clear,NaN,clear,clear,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5875,consider,NaN,consider,NaN,unidentified,2017-10-31 21:06:00,clear,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5876,consider,NaN,consider,NaN,unidentified,2017-10-31 21:10:00,clear,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5877,clear,clear,clear,clear,clear,2017-10-31 21:11:00,clear,clear,clear,clear,clear,NaN,clear,clear
5878,clear,clear,clear,clear,clear,2017-10-31 21:46:00,clear,clear,clear,NaN,NaN,NaN,clear,clear


In [11]:
# Set 'created_at' as the index
document_filtered.set_index('created_at', inplace=True)

# Group by month and apply value_counts for each column
monthly_stats = document_filtered.resample('M').apply(lambda x: x.apply(lambda y: y.value_counts())).unstack(fill_value=0)

# Reset the index to make 'created_at' a column again
monthly_stats.reset_index(inplace=True)



In [12]:
monthly_stats

created_at  result                       visual_authenticity_result  \
               clear consider unidentified                      clear   
0 2017-05-31    56.0      2.0          0.0                       58.0   
1 2017-06-30   267.0      9.0          NaN                      272.0   
2 2017-07-31  1013.0    120.0          NaN                     1019.0   
3 2017-08-31  1017.0    211.0          NaN                     1034.0   
4 2017-09-30   901.0    352.0          NaN                     1012.0   
5 2017-10-31  1152.0    780.0          NaN                     1499.0   

                        image_integrity_result                        ...  \
  consider unidentified                  clear consider unidentified  ...   
0      NaN          0.0                   58.0      NaN          0.0  ...   
1      3.0          NaN                  275.0      1.0          NaN  ...   
2     18.0          NaN                 1037.0     96.0          NaN  ...   
3     29.0          NaN                 1051.0    177.0          NaN  ...   
4     21.0          NaN                  930.0    323.0          NaN  ...   
5     26.0          NaN                 1192.0    740.0          NaN  ...   

  data_consistency_result data_comparison_result                        \
             unidentified                  clear consider unidentified   
0                     0.0                   56.0      2.0          0.0   
1                     NaN                   24.0      3.0          NaN   
2                     NaN                    NaN      NaN          NaN   
3                     NaN                    NaN      NaN          NaN   
4                     NaN                    NaN      NaN          NaN   
5                     NaN                    NaN      NaN          NaN   

  police_record_result                       compromised_document_result  \
                 clear consider unidentified                       clear   
0                 49.0      NaN          0.0                         NaN   
1                238.0      NaN          NaN                         NaN   
2               1008.0      NaN          NaN                         NaN   
3               1026.0      NaN          NaN                         NaN   
4                986.0      NaN          NaN                       133.0   
5               1484.0      NaN          NaN                      1391.0   

                         
  consider unidentified  
0      NaN          0.0  
1      NaN          NaN  
2      NaN          NaN  
3      NaN          NaN  
4      NaN          NaN  
5      NaN          NaN  

[6 rows x 40 columns]